In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from warnings import warn
from typing import Callable, Optional

# Defining processes

In [ ]:
class CountingProcess:
    def __init__(self,
                 T: float,
                 claim_law: Optional[Callable[[float], float]] = None,
                 u: float = 1.0,
                 p: float = 1.0,
                 process_name: str = "Counting process",
                 seed: int = None):
        self.res = 100
        if seed is None:
            seed = np.random.randint(0, 2**32)
            print(f"Seed: {seed}")
        np.random.seed(seed)
        if T <= 0:
            raise ValueError("T must be strictly positive")
        self.T = T
        self.name = process_name
        self.events = self.generate_events()
        self.N = len(self.events)
        self.claim_law = claim_law if claim_law is not None else lambda: np.random.uniform(0,1)
        self.claims = self.generate_claims()
        self.u = u
        self.p = p

    def generate_events(self):
        raise NotImplementedError("The method 'generate_events' must be implemented in a subclass")

    def generate_intensity(self):
        raise NotImplementedError("The method 'generate_intensity' must be implemented in a subclass")

    def get_mean_and_variance(self, t, variable="process"):
        raise NotImplementedError("The method 'get_mean_and_variance' must be implemented in a subclass")

    def generate_claims(self):
        return np.array([self.claim_law() for _ in range(self.N)])

    def generate_risk(self):
        time = [0.]
        resources = [self.u]
        for (t,c) in zip(self.events, self.claims):
            resources.append(resources[-1] + self.p*(t - time[-1]))  # add the new resource
            resources.append(resources[-1] - c)  # remove the sinister
            time += [t, t]
        resources.append(resources[-1] + self.p*(self.T - time[-1]))
        time.append(self.T)
        return time, resources

    def _events_with_limits(self):
        return [0.] + self.events + [self.T]

    def _theory_on_ax(self, ax, variable="process"):
        try:
            x = np.linspace(0., self.T, self.res)
            mean, var = self.get_mean_and_variance(x, variable=variable)
            ax.plot(x, mean, color="green")
            ax.fill_between(x, mean - 1.96*np.sqrt(var), mean + 1.96*np.sqrt(var), color="green", alpha=0.2)
        except NotImplementedError:
            warn(f"The method 'get_mean_and_variance' is not implemented for variable '{variable}'")

    def _process_on_ax(self, ax, plot_theory=False):
        ax.step(self._events_with_limits(), list(range(self.N + 1)) + [self.N], where="post")
        if plot_theory:
            self._theory_on_ax(ax, variable="process")
        ax.vlines(self.events, *ax.get_ylim(), color="red", linewidth=0.8, linestyle="dashed")
        ax.set_xlim(0, self.T)
        ax.set_title(self.name)
        ax.set_xlabel("Time")
        ax.set_ylabel("$N(t)$")
        ax.grid()

    def _intensity_on_ax(self, ax, plot_theory=False):
        absintens, intens = self.generate_intensity()
        ax.plot(absintens, intens)
        if plot_theory:
            self._theory_on_ax(ax, variable="intensity")
        ax.vlines(self.events, *ax.get_ylim(), color="red", linewidth=0.8, linestyle="dashed")
        ax.set_xlim(0, self.T)
        ax.set_title(self.name + " intensity")
        ax.set_xlabel("Time")
        ax.set_ylabel("$\lambda(t)$")
        ax.grid()

    def _risk_on_ax(self, ax, plot_theory=False):
        absrisk, risk = self.generate_risk()
        ax.plot(absrisk, risk)
        if plot_theory:
            self._theory_on_ax(ax, variable="risk")
        ax.hlines(0, *ax.get_xlim(), color="blue", linestyle="dashed")
        ax.vlines(self.events, *ax.get_ylim(), color="red", linewidth=0.8, linestyle="dashed")
        ruin = np.array([t for (i,(t,r)) in enumerate(zip(absrisk[1:], risk[1:])) if r < 0 and risk[i] > 0])
        ax.scatter(ruin, np.zeros_like(ruin), color="red", zorder=10)
        ax.set_xlim(0, self.T)
        ax.set_title(self.name + f" risk (u={self.u}, p={self.p})")
        ax.set_xlabel("Time")
        ax.set_ylabel("Resources")
        ax.grid()

    def plot_process(self, plot_theory=False, dim=(10,6), save=None):
        fig, ax = plt.subplots(figsize=dim)
        self._process_on_ax(ax, plot_theory)
        if save is not None:
            fig.savefig(save, transparent=True)
        plt.show()

    def plot_intensity(self, plot_theory=False, dim=(10,6), save=None):
        fig, ax = plt.subplots(figsize=dim)
        self._intensity_on_ax(ax, plot_theory)
        if save is not None:
            fig.savefig(save, transparent=True)
        plt.show()

    def plot_risk(self, plot_theory=False, dim=(10,6), save=None):
        fig, ax = plt.subplots(figsize=dim)
        self._risk_on_ax(ax, plot_theory)
        if save is not None:
            fig.savefig(save, transparent=True)
        plt.show()

    def plot_process_and_risk(self, plot_theory=False, dim=(10,8), save=None):
        fig, ax = plt.subplots(figsize=dim)
        self._process_on_ax(ax, plot_theory)
        self._risk_on_ax(ax, plot_theory)
        if save is not None:
            fig.savefig(save, transparent=True)
        plt.show()

    def plot_process_and_intensity(self, plot_theory=False, dim=(10,8), save=None):
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=dim)
        self._process_on_ax(ax1, plot_theory=plot_theory)
        self._intensity_on_ax(ax2, plot_theory=plot_theory)
        fig.tight_layout()
        if save is not None:
            fig.savefig(save, transparent=True)
        plt.show()

    def plot_process_and_intensity_and_risk(self, plot_theory=False, dim=(10,10), save=None):
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=dim)
        self._process_on_ax(ax1, plot_theory)
        self._intensity_on_ax(ax2, plot_theory)
        self._risk_on_ax(ax3, plot_theory)
        fig.tight_layout()
        if save is not None:
            fig.savefig(save, transparent=True)
        plt.show()

In [ ]:
class HawkesProcess(CountingProcess):
    def __init__(self,
                 mu: float,
                 alpha: float,
                 beta: float,
                 name: str = "Hawkes process",
                 **kwargs):
        if alpha <  0 or beta < alpha:
            raise ValueError(f"We must have 0 < alpha < beta (alpha={alpha} and beta={beta})")
        self.mu = mu
        self.alpha = alpha
        self.beta = beta
        super().__init__(process_name=name, **kwargs)

    def get_mean_and_variance(self, t, variable="process"):
        diff = self.beta - self.alpha
        mu_inf = self.beta*self.mu/diff
        diff_mu = self.mu - mu_inf
        if variable == "process":
            mean = mu_inf*t + diff_mu/diff * (1 - np.exp(-diff*t))
            var = self.beta**2*mu_inf/(diff**2)*t + \
                  self.alpha**2*(2*self.mu - mu_inf)/(2*diff**3) * (1 - np.exp(-2*diff*t)) - \
                  2*self.alpha*self.beta*diff_mu/(diff**2) * t * np.exp(-diff*t) + \
                  ( (self.beta+self.alpha)/(diff**2) * diff_mu - 2*self.alpha*self.beta/(diff**3) * mu_inf ) * (1 - np.exp(-diff*t))
        elif variable == "intensity":
            mean = mu_inf + diff_mu * np.exp(-diff*t)
            var = self.alpha**2*mu_inf/(2*diff) + self.alpha**2*diff_mu/diff*np.exp(-diff*t) - self.alpha**2*(2*self.mu-mu_inf)/(2*diff)*np.exp(-2*diff*t)
        elif variable == "risk":
            claim_mean = np.mean([self.claim_law() for _ in range(100000)])
            mean_process, _ = self.get_mean_and_variance(t, variable="process")
            mean = self.u + self.p*t - claim_mean * mean_process
            var = 0
            warn("Risk variance is not implemented for Hawkes process", stacklevel=2)
        else:
            raise NotImplementedError(f"Not implemented yet for variable {variable}")
        return mean, np.clip(var, 0, np.inf)

    def phi(self, t):
        return self.alpha * np.exp(-self.beta * t)

    def lamb(self, t, tau):
        return self.mu + sum([self.phi(t-tk) for tk in tau])

    def generate_events(self):
        Tau = []
        s = 0
        while s < self.T:
            lambda_bar = self.lamb(s, Tau)
            w = np.random.exponential(1/lambda_bar)
            s += w
            D = np.random.uniform(0, lambda_bar)
            if D <= self.lamb(s, Tau):
                Tau.append(s)
        if Tau[-1] <= self.T:
            return Tau
        else:
            return Tau[:-1]

    def generate_intensity(self):
        x = np.array([0, self.events[0]])
        intensity = np.array([self.mu, self.mu])
        pointsT = self.events + [self.T]
        for i in range(0, len(pointsT)-1):
            p1, p2 = pointsT[i:i+2]
            local_x = np.linspace(p1, p2, self.res)
            local_intensity = [self.lamb(xi, pointsT[:i+1]) for xi in local_x]
            x = np.append(x, local_x)
            intensity = np.append(intensity, local_intensity)
        return x, intensity

h = HawkesProcess(mu=1.2, alpha=0.6, beta=0.8, T=20, u=1, p=1, seed=120)
h.plot_process_and_intensity_and_risk(plot_theory=True)

In [ ]:
class PoissonProcess(CountingProcess):
    def __init__(self,
                 lamb: float,
                 name: str = "Poisson process",
                 **kwargs):
        if lamb <= 0:
            raise ValueError(f"lambda must be strictly positive (lambda={lamb})")
        self.lamb = lamb
        super().__init__(process_name=name, **kwargs)

    def get_mean_and_variance(self, t, variable="process"):
        raise NotImplementedError(f"Not implemented yet")

    def generate_events(self):
        Tau = []
        s = 0
        while s < self.T:
            u = np.random.uniform(0,1)
            w = -np.log(u)/self.lamb
            s += w
            Tau.append(s)
        return Tau

    def generate_intensity(self):
        x = [0, self.T]
        intensity = [self.lamb, self.lamb]
        return x, intensity

p = PoissonProcess(lamb=2, T=20, seed=120)
p.plot_process_and_intensity_and_risk()

In [ ]:
class NonHomogeneousPoissonProcess(CountingProcess):
    def __init__(self,
                 lamb: Callable[[float], float],
                 name: str = "Inhomogeneous Poisson process",
                 **kwargs):
        self.lamb = lamb
        super().__init__(process_name=name, **kwargs)
        if not all([self.lamb(x) >= 0 for x in np.arange(0, self.T, self.res)]):
            raise ValueError(f"lambda must be strictly positive (lambda={lamb})")

    def get_mean_and_variance(self, t, variable="process"):
        raise NotImplementedError(f"Not implemented yet")

    def generate_events(self):
        M = max(self.generate_intensity()[1])
        Tau = []
        s = 0
        while s < self.T:
            w = np.random.exponential(1/M)
            s += w
            D = np.random.uniform(0, M)
            if D <= self.lamb(s):
                Tau.append(s)
        if Tau[-1] <= self.T:
            return Tau
        else:
            return Tau[:-1]

    def generate_intensity(self):
        x = np.linspace(0, self.T, self.res)
        intensity = [self.lamb(xi) for xi in x]
        return x, intensity

nhp = NonHomogeneousPoissonProcess(lamb=lambda x: (.8*np.cos(x-8)+1)**4+1, T=8, seed=1177971048)
nhp.plot_process_and_intensity_and_risk(save="./figures/nh_poisson_introduction.png")

# Varying $p$ and $u$

In [ ]:
n = 10
max_u = 5
max_p = 5
u = np.linspace(0, max_u, n)
p = np.linspace(0, max_p, n)

R_hp = []
R = []
uu, pp = [], []
from tqdm.notebook import tqdm
np.random.seed(10)
for i in tqdm(range(0, u.shape[0])):
    for j in range(0, p.shape[0]):
        for k in range(0,30):
            h = HawkesProcess(mu=2, alpha=0.6, beta=.8, T=20, u=u[i], p=p[j], seed=None)
            h.claims = h.generate_claims()

            xvalues_hp, risk_hp = h.generate_risk()

            ruin_hp = np.array([t for (t,r) in zip(xvalues_hp, risk_hp) if r < 0])

            if len(ruin_hp) != 0:
                R_hp.append(ruin_hp[0])

        R.append(np.mean(R_hp))
        uu.append(u[i])
        pp.append(p[j])

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
p = ax.imshow(np.array(R).reshape((n,n)), origin="lower", extent=[0, max_p, 0, max_u])
plt.colorbar(p)
ax.set_ylabel("u")
ax.set_xlabel("p")
fig.savefig("./figures/vary_p_and_u.pdf", dpi=100, bbox_inches="tight")
plt.show()

# Example cac40

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('cac40.csv',parse_dates=["Date"],infer_datetime_format=True)
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(df["Date"], df["Close"])
ax.set_title("CAC 40 stock market evolution")
ax.set_xlabel("Date")
ax.set_ylabel("Value")
ax.grid()
fig.savefig("./figures/cac40.pdf", dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
diff = np.diff(df["Close"])
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(df["Date"], df["Close"])
events = np.argwhere(diff < -150).ravel().astype(float)
ax.vlines(df["Date"][events], *ax.get_ylim(), color="red", zorder=10, linestyles="dashed", linewidth=1, alpha=.5)
ax.set_title("Evolution of CAC40 (highlighting drops greater than 150)")
ax.set_xlabel("Date")
ax.set_ylabel("Value")
ax.grid()
fig.savefig("./figures/cac40_events.pdf", dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
from tick.hawkes import HawkesExpKern

In [ ]:
betas = np.logspace(-4, 1, 100)  # as tick does not fit beta, we do it manually
scores = []
for beta in betas:
    process = HawkesExpKern(beta).fit([events])
    scores.append(process.score([events]))
fig, ax = plt.subplots(figsize=(10,5))
ax.semilogx(betas, scores)  # plot log-likelihood as a function of beta
ax.set_title("Log-likelihood as a function of beta")
ax.set_xlabel("Beta")
ax.set_ylabel("Log-likelihood")
ax.grid()
fig.savefig("./figures/beta_likelihood.pdf", dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
best_index = np.argmax(scores)
best_beta = betas[best_index]
print(f"Best beta: {best_beta}")
process = HawkesExpKern(best_beta).fit([events])
process.fit([events])
mu = process.coeffs[0]
alpha = process.coeffs[1]*best_beta  # as the alpha of our model is not the same as the tick one (our_alpha = tick_alpha*beta)
print(f"Best mu: {mu}")
print(f"Best alpha: {alpha}")
print(f"Model log-likelihood: {process.score([events])}")

In [ ]:
intens, x_steps = process.estimated_intensity([events], 1, end_time=len(df))
integer = np.argwhere(np.diff(x_steps) != 0).ravel()  # fix a bug with the dimensions (same for the following df["Date"][:-2])
intens = intens[0][integer]
x_steps = x_steps[integer]
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(df["Date"][:-2], intens)
ax.vlines(df["Date"][events], *ax.get_ylim(), color="red", zorder=10, linestyles="dashed", linewidth=1, alpha=.3)
ax.set_title("Intensity of the fitted process")
ax.set_xlabel("Date")
ax.set_ylabel("Intensity")
ax.grid()
fig.savefig("./figures/cac40_intensity.pdf", dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
# Visual goodness test based on https://pat-laub.github.io/pdfs/honours_thesis.pdf
cum_intens = np.cumsum(intens)  # compute cumulative intensity
fig, ax = plt.subplots(figsize=(15,5))
ax.plot(df["Date"][:-2], cum_intens)
ax.vlines(df["Date"][events], *ax.get_ylim(), color="red", alpha=.2, zorder=10, linestyle="dashed")
ax.set_title("Cumulated intensity of the fitted process")
ax.set_xlabel("Date")
ax.set_ylabel("Cumulated intensity")
ax.grid()
fig.savefig("./figures/cac40_cum_intensity.pdf", dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
from scipy.stats import expon
def cum_intens_func(t):
    return cum_intens[np.searchsorted(x_steps, t-1)]
events_transformed = cum_intens_func(events)  # this should be a Poisson process with intensity 1
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(20,5))
ax1.scatter(events_transformed, np.arange(1, len(events_transformed)+1))
ax1.plot([0, events_transformed.max()], [0, events_transformed.max()], color="red", linestyle="dashed")  # this should align on the line y=x (goodness of fit)
ax1.set_title("Q-Q plot for validating the Poisson process assertion (transformed Hawkes process)")
ax1.set_xlabel("Empirical quantiles")
ax1.set_ylabel("Theoretical quantiles")
ax2.scatter(expon.cdf(np.diff(events_transformed)), expon.cdf(np.diff(np.roll(events_transformed, 1))))  # this should be uniformly distributed (independence of arrival times)
ax2.set_title(r"Distribution of the points ($T_i, T_{i-1}$)")
ax2.set_xlabel("$T_i$")
ax2.set_ylabel("$T_{i-1}$")
ax1.grid()
ax2.grid()
fig.savefig("./figures/cac40_qq_plot.pdf", dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
# same plots but directly for the Hawkes process (to show that it doesn't fit)
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(20,5))
ax1.scatter(events, np.arange(1, len(events)+1))
ax1.set_title("Q-Q plot for the not transformed Hawkes process")
ax1.set_xlabel("Empirical quantiles")
ax1.set_ylabel("Theoretical quantiles")
ax2.scatter(expon.cdf(np.diff(events)), expon.cdf(np.diff(np.roll(events, 1))))
ax2.set_title("Distribution of the points (Zi,Zi-1)")
ax2.set_title(r"Distribution of the points ($T_i, T_{i-1}$)")
ax2.set_xlabel("$T_i$")
ax2.set_ylabel("$T_{i-1}$")
ax1.grid()
ax2.grid()
fig.savefig("./figures/cac40_qq_plot_hawkes.pdf", dpi=100, bbox_inches='tight')
plt.show()